In [248]:
import pandas as pd
import requests
import json
from pytz import timezone
from datetime import datetime, timedelta

# Main function to retrieve data from Rapid API (AeroDataBox) and send it to SQL
def retrieve_and_send_data():
  connection_string = create_connection_string()
  airports_df = fetch_airports_data(connection_string)
  flights_df = icao_flights(airports_df)
  store_flights_data(flights_df, connection_string)
  return "Data has been updated"

# Function to create a connection string to save data in SQL database
def create_connection_string():
  schema = "NAME_OF_YOUR_SQL_DATABASE"
  host = "11.11.111.111"
  user = "root"
  password = "YOUR_PASSWORD"
  port = 3306
  return f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

# Function to get airports data (airport name,icao) from SQL
def fetch_airports_data(connection_string):
  return pd.read_sql("airports", con=connection_string)

# Function to get icao data (airport code) from API
def icao_flights(airports_df):
    berlin_timezone = timezone('Europe/Berlin')
    tomorrow_datetime = datetime.now(berlin_timezone) + timedelta(days=1)
    flight_items=[]
    flight_items_2=[]
    #for the first 12 hours (from 00:00 till 11:59)
    for _, airport in airports_df.iterrows():
            airport_icao = airport['airport_icao']
            tomorrow_start_of_day = berlin_timezone.localize(datetime(tomorrow_datetime.year, tomorrow_datetime.month, tomorrow_datetime.day, 0, 0))
            tomorrow_midday = berlin_timezone.localize(datetime(tomorrow_datetime.year, tomorrow_datetime.month, tomorrow_datetime.day, 11, 59))
            start_time = tomorrow_start_of_day.strftime("%Y-%m-%dT%H:%M")
            end_time = tomorrow_midday.strftime("%Y-%m-%dT%H:%M")
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{airport_icao}/{start_time}/{end_time}"
            querystring = {"direction":"Arrival"}
            headers = {
            	"x-rapidapi-key": "YOUR_API_KEY",
            	"x-rapidapi-host": "aerodatabox.p.rapidapi.com"
            }
            response = requests.get(url, headers=headers, params=querystring)
            flights_new=response.json()
            for i in flights_new['arrivals']:
                flight_item = {
                            "flight_num": i.get('number'),
                            "departure_icao": i['movement']['airport'].get('icao'),
                            "arrival_icao": airport_icao,
                            "arrival_time": i['movement']['scheduledTime'].get('local'),
                }
                flight_items.append(flight_item)
    flights_df_1 = pd.DataFrame(flight_items)

    for _, airport in airports_df.iterrows():
            airport_icao = airport['airport_icao']
            tomorrow_midday_2 = berlin_timezone.localize(datetime(tomorrow_datetime.year, tomorrow_datetime.month, tomorrow_datetime.day, 12, 0))
            tomorrow_night = berlin_timezone.localize(datetime(tomorrow_datetime.year, tomorrow_datetime.month, tomorrow_datetime.day, 23, 59))
            start_time_2 = tomorrow_midday_2.strftime("%Y-%m-%dT%H:%M")
            end_time_2 = tomorrow_night.strftime("%Y-%m-%dT%H:%M")
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{airport_icao}/{start_time_2}/{end_time_2}"
            querystring = {"direction":"Arrival"}
            headers = {
            	"x-rapidapi-key": "YOUR_API_KEY",
            	"x-rapidapi-host": "aerodatabox.p.rapidapi.com"
            }
            response_2 = requests.get(url, headers=headers, params=querystring)
            flights_2=response_2.json()
            for i in flights_2['arrivals']:
                flight_item_2 = {
                            "flight_num": i.get('number'),
                            "departure_icao": i['movement']['airport'].get('icao'),
                            "arrival_icao": airport_icao,
                            "arrival_time": i['movement']['scheduledTime'].get('local'),
                }
                flight_items_2.append(flight_item_2)
    flights_df_2 = pd.DataFrame(flight_items_2)
    flights_df = pd.concat([flights_df_1,flights_df_2],axis=0)
    flights_df["arrival_time"] = pd.to_datetime(flights_df["arrival_time"]) 
    flights_df["arrival_time"] = flights_df["arrival_time"].dt.strftime("%Y-%m-%dT%H:%M")
    flights_df["arrival_time"] = pd.to_datetime(flights_df["arrival_time"])
    return flights_df

    
#Function to store flights data as an SQL database
def store_flights_data(flights_df, connection_string):
  flights_df.to_sql('flights',
                    if_exists='append',
                    con=connection_string,
                    index=False)



In [250]:
retrieve_and_send_data()

'Data has been updated'

In [ ]:
#libraries used (for requirements.txt on GCP)
functions-framework
SQLAlchemy
PyMySQL
pandas
requests
pytz